In [ ]:
# Загрузим необходимые бибилиотеки
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import json
import lxml
import os
import re
import time
from tqdm.notebook import tqdm
from google.colab import files

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Получаю список ссылок на страницы с книгами с сайта loyalbooks.com/Top_100, проходя по страницам с 1 по 384

In [ ]:
books_refs = []
for page in tqdm(range(385)):
    response = requests.get(f"https://www.loyalbooks.com/Top_100/{page}")

    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')

        table = soup.find('table', class_='layout2-blue')
        try:
            links = table.find_all('a', href=True)
        except:
            pass
        for link in links:
            print(link['href'])
            books_refs.append(link['href'])
    else:
        print("Не удалось получить страницу")

  0%|          | 0/385 [00:00<?, ?it/s]

Выходные данные были обрезаны до нескольких последних строк (5000).
/book/Childhood-of-Distinguished-Women-by-Bower
/book/Childhood-of-Distinguished-Women-by-Bower
/book/Fore-by-Charles-Emmett-Van-Loan
/book/Fore-by-Charles-Emmett-Van-Loan
/book/Other-Fellow-by-Francis-Hopkinson-Smith
/book/Other-Fellow-by-Francis-Hopkinson-Smith
/book/Further-Adventures-of-ONeill-in-Holland
/book/Further-Adventures-of-ONeill-in-Holland
/book/Story-of-Fifty-Seven-Cents-and-Others
/book/Story-of-Fifty-Seven-Cents-and-Others
/book/Land-of-Lure-by-Elliott-Smith
/book/Land-of-Lure-by-Elliott-Smith
/book/Aunt-Hannah-and-Seth-by-James-Otis
/book/Aunt-Hannah-and-Seth-by-James-Otis
/book/For-Johns-Sake-by-Annie-Frances-Perram
/book/For-Johns-Sake-by-Annie-Frances-Perram
/book/Cornflower-and-Other-Poems-by-Jean-Blewett
/book/Cornflower-and-Other-Poems-by-Jean-Blewett
/book/Dear-Little-Girl-by-Amy-Ella-Blanchard
/book/Dear-Little-Girl-by-Amy-Ella-Blanchard
/book/Rub%C3%A1iy%C3%A1t-of-a-Bachelor-by-Helen-Rowland


Удаляю дубликаты из полученного списка

In [ ]:
books_refs = list(set(books_refs))
len(books_refs)
books_refs

['/book/een-nagelaten-bekentenis-door-marcellus-emants',
 '/book/Joe-Tildens-Recipes-for-Epicures',
 '/book/Builders-by-Ellen-Anderson-Gholson-Glasgow',
 '/book/queen-victoria-by-lytton-strachey',
 '/book/Helden-des-Dampfes',
 '/book/Dorothy-on-a-House-Boat-by-Evelyn-Raymond',
 '/book/How-to-Care-for-the-Insane-by-William-D-Granger',
 '/book/True-Vine',
 '/book/the-servile-state-by-hillaire-belloc',
 '/book/Great-Mogul-by-Louis-Tracy',
 '/book/greener-than-you-think-by-ward-moore',
 '/book/History-of-the-Caliph-Vathek',
 '/book/Diary-from-Dixie',
 '/book/a-few-figs-from-thistles-by-edna-st-vincent-millay',
 '/book/Donalblane-of-Darien-by-J-Macdonald-Oxley',
 '/book/At-the-Post-by-H-L-Gold',
 '/book/Drama-of-Glass-by-Kate-Field',
 '/book/Instruction-of-Ptah-Hotep-and-the-Instruction',
 '/book/Lives-of-Girls-Who-Became-Famous',
 '/book/adam-bede-by-george-eliot',
 '/book/the-beasts-of-tarzan',
 '/book/Taming-of-the-Shrew',
 '/book/Ashiel-Mystery-A-Detective-Story',
 '/book/Toppletons-Cli

Оставлю только название книги

In [ ]:
l = []
for ref in books_refs:
    l.append(ref[6:])
l

['een-nagelaten-bekentenis-door-marcellus-emants',
 'Joe-Tildens-Recipes-for-Epicures',
 'Builders-by-Ellen-Anderson-Gholson-Glasgow',
 'queen-victoria-by-lytton-strachey',
 'Helden-des-Dampfes',
 'Dorothy-on-a-House-Boat-by-Evelyn-Raymond',
 'How-to-Care-for-the-Insane-by-William-D-Granger',
 'True-Vine',
 'the-servile-state-by-hillaire-belloc',
 'Great-Mogul-by-Louis-Tracy',
 'greener-than-you-think-by-ward-moore',
 'History-of-the-Caliph-Vathek',
 'Diary-from-Dixie',
 'a-few-figs-from-thistles-by-edna-st-vincent-millay',
 'Donalblane-of-Darien-by-J-Macdonald-Oxley',
 'At-the-Post-by-H-L-Gold',
 'Drama-of-Glass-by-Kate-Field',
 'Instruction-of-Ptah-Hotep-and-the-Instruction',
 'Lives-of-Girls-Who-Became-Famous',
 'adam-bede-by-george-eliot',
 'the-beasts-of-tarzan',
 'Taming-of-the-Shrew',
 'Ashiel-Mystery-A-Detective-Story',
 'Toppletons-Client-by-John-Kendrick-Bangs',
 'Butterfly-on-the-Wheel-by-Gull',
 'Girl-Who-Sat-By-The-Ashes',
 'School-History-of-the-Great-War',
 'Persian-Hidd

Получаю, список ссылок на тексты книг

In [ ]:
links = []
for url in tqdm(l):
    response = requests.get(f'https://www.loyalbooks.com/book/{url}')
    html_content = response.text
    soup = BeautifulSoup(html_content, 'html.parser')

    a_tags = soup.find_all('a', class_='download', href=lambda href: href and href.endswith('.txt'))
    hrefs = [a['href'] for a in a_tags]
    links.append(hrefs)
    print(f'Найденная ссылка: {hrefs}')

for link in links:
    print(link)

  0%|          | 0/5597 [00:00<?, ?it/s]

Выходные данные были обрезаны до нескольких последних строк (5000).
['/download/text/Streets-of-Ascalon-by-Robert-W-Chambers.txt']
['/download/text/Oedipus-Trilogy-by-Sophocles.txt']
['/download/text/My-Little-Sister-by-Elizabeth-Robins.txt']
['/download/text/Penelopes-English-Experiences-by-Wiggin.txt']
['/download/text/Messenger-by-Elizabeth-Robins.txt']
['/download/text/Hermann-Und-Dorothea.txt']
[]
[]
[]
['/download/text/Youth-Leo-Tolstoy.txt']
['/download/text/Life-Adventures-Piracies-of-Captain-Singleton.txt']
['/download/text/Daddys-Girl-by-L-T-Meade.txt']
['/download/text/The-Return-of-Sherlock-Holmes-by-Sir-Arthur-Conan-Doyle.txt']
['/download/text/Bible-Douay-Rheims.txt']
['/download/text/The-Elusive-Pimpernel-by-Baroness-Emmuska-Orczy.txt']
['/download/text/Great-Stone-Face.txt']
['/download/text/South-African-Memories-by-Wilson.txt']
[]
['/download/text/Dress-Design-by-Talbot-Hughes.txt']
['/download/text/The-Hampstead-Mystery-by-John-R-Watson-and-Arthur-J-Rees.txt']
['/dow

Сделал плоский список из полученного и обрезал строки, чтобы получить список с именами файлов

In [ ]:
def flatten(nested_list):
    flat_list = []
    for item in nested_list:
        if isinstance(item, list):
            flat_list.extend(flatten(item))
        else:
            flat_list.append(item)
    return flat_list


lis = flatten(links)
print(lis)
l2 = []
for ref in lis:
    l2.append(ref[15:])
l2

['/download/text/Joe-Tildens-Recipes-for-Epicures.txt', '/download/text/Builders-by-Ellen-Anderson-Gholson-Glasgow.txt', '/download/text/Queen-Victoria-by-Lytton-Strachey.txt', '/download/text/Dorothy-on-a-House-Boat-by-Evelyn-Raymond.txt', '/download/text/How-to-Care-for-the-Insane-by-William-D-Granger.txt', '/download/text/Great-Mogul-by-Louis-Tracy.txt', '/download/text/Greener-Than-You-Think-by-Ward-Moore.txt', '/download/text/History-of-the-Caliph-Vathek.txt', '/download/text/A-Few-Figs-from-Thistles-by-Edna-St-Vincent-Millay.txt', '/download/text/Donalblane-of-Darien-by-J-Macdonald-Oxley.txt', '/download/text/At-the-Post-by-H-L-Gold.txt', '/download/text/Drama-of-Glass-by-Kate-Field.txt', '/download/text/Instruction-of-Ptah-Hotep-and-the-Instruction.txt', '/download/text/Lives-of-Girls-Who-Became-Famous.txt', '/download/text/Adam-Bede-by-George-Eliot.txt', '/download/text/The-Beasts-of-Tarzan.txt', '/download/text/Taming-of-the-Shrew.txt', '/download/text/Ashiel-Mystery-a-Detecti

['Joe-Tildens-Recipes-for-Epicures.txt',
 'Builders-by-Ellen-Anderson-Gholson-Glasgow.txt',
 'Queen-Victoria-by-Lytton-Strachey.txt',
 'Dorothy-on-a-House-Boat-by-Evelyn-Raymond.txt',
 'How-to-Care-for-the-Insane-by-William-D-Granger.txt',
 'Great-Mogul-by-Louis-Tracy.txt',
 'Greener-Than-You-Think-by-Ward-Moore.txt',
 'History-of-the-Caliph-Vathek.txt',
 'A-Few-Figs-from-Thistles-by-Edna-St-Vincent-Millay.txt',
 'Donalblane-of-Darien-by-J-Macdonald-Oxley.txt',
 'At-the-Post-by-H-L-Gold.txt',
 'Drama-of-Glass-by-Kate-Field.txt',
 'Instruction-of-Ptah-Hotep-and-the-Instruction.txt',
 'Lives-of-Girls-Who-Became-Famous.txt',
 'Adam-Bede-by-George-Eliot.txt',
 'The-Beasts-of-Tarzan.txt',
 'Taming-of-the-Shrew.txt',
 'Ashiel-Mystery-a-Detective-Story.txt',
 'Toppletons-Client-by-John-Kendrick-Bangs.txt',
 'Butterfly-on-the-Wheel-by-Gull.txt',
 'School-History-of-the-Great-War.txt',
 'Castles-and-Chateaux-of-Old-Touraine-and-the-Loire.txt',
 'Overland-Through-Asia-by-Thomas-Wallace-Knox.txt'

Скачиваю и сохраняю txt-файлы по полученному списку

In [ ]:
def download_file(url, local_filename):
    with requests.get(url, stream=True) as r:
        r.raise_for_status()
        with open(local_filename, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192):
                f.write(chunk)

for url in tqdm(l2):
    download_file(f'https://www.loyalbooks.com//download//text//{url}', f'/content/drive/My Drive/books_/{url}')
    print(f'Файл сохранён')

  0%|          | 1/4329 [00:00<15:18,  4.71it/s]

Файл сохранён


  0%|          | 2/4329 [00:00<21:12,  3.40it/s]

Файл сохранён


  0%|          | 3/4329 [00:00<22:27,  3.21it/s]

...

Файл сохранён


100%|█████████▉| 4328/4329 [20:34<00:00,  3.40it/s]

Файл сохранён


100%|██████████| 4329/4329 [20:35<00:00,  3.51it/s]

Файл сохранён


Создам датафрейм с именами скаченных файлов с текстами книг, в имени файла присутствует ФИО автора и название книги

In [ ]:
file_list = os.listdir('/content/drive/My Drive/books_/')
df2 = pd.DataFrame(file_list, columns=['File Names'])
df2

,File Names
0,Fairy-Tales-by-the-Brothers-Grimm.txt
1,Invisible-Man-by-H-G-Wells.txt
2,King-Solomons-Mines-by-Haggard.txt
3,A-Christmas-Carol-by-Charles-Dickens.txt
4,Mystery-of-the-Yellow-Room-by-Gaston-Leroux.txt
...,...
4308,Wreck-of-the-Golden-Mary-by-Charles-Dickens.txt
4309,Moon-Metal-by-Garrett-P-Serviss.txt
4310,Within-the-Rim-and-Other-Essays-by-Henry-James...
4311,Rough-Hewn-by-Dorothy-Canfield-Fisher.txt
